In [1]:
import torch
import torch.nn as nn

In [2]:
class Model(nn.Module):
    def __init__(self, n_input_features):
        super(Model, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)

    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

model = Model(n_input_features=6)

In [3]:
FILE = "model.pth"
torch.save(model, FILE)

In [4]:
model = torch.load(FILE)
model.eval()

Model(
  (linear): Linear(in_features=6, out_features=1, bias=True)
)

In [5]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[-0.2840,  0.2932, -0.3852, -0.0927,  0.3099,  0.3473]],
       requires_grad=True)
Parameter containing:
tensor([0.2393], requires_grad=True)


# Another way

In [6]:
import os 
os.remove("/kaggle/working/model.pth")

In [7]:
class Model(nn.Module):
    def __init__(self, n_input_features):
        super(Model, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)

    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

model = Model(n_input_features=6)

In [8]:
FILE = "model.pth"
torch.save(model.state_dict(), FILE)

In [9]:
loaded_model = Model(n_input_features=6)
loaded_model.load_state_dict(torch.load(FILE))
loaded_model.eval()

Model(
  (linear): Linear(in_features=6, out_features=1, bias=True)
)

In [10]:
for param in loaded_model.parameters():
    print(param)

Parameter containing:
tensor([[ 0.3499,  0.2772,  0.2277,  0.1958,  0.2954, -0.3927]],
       requires_grad=True)
Parameter containing:
tensor([-0.2384], requires_grad=True)


# Saving during training MB

In [11]:
import os 
#os.remove("/kaggle/working/checkpoint.pth")

In [12]:
class Model(nn.Module):
    def __init__(self, n_input_features):
        super(Model, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)

    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

model = Model(n_input_features=6)

In [13]:
learning_rate =0.01
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)
print(optimizer.state_dict())

{'state': {}, 'param_groups': [{'lr': 0.01, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'maximize': False, 'params': [0, 1]}]}


In [14]:
checkpoint = {
    "epoch": 90,
    "model_state":model.state_dict(),
    "optim_state":optimizer.state_dict() 
}

torch.save(checkpoint, "checkpoint.pth")

In [15]:
#Load checkpoint
loaded_checkpoint = torch.load("checkpoint.pth")
epoch = loaded_checkpoint["epoch"]

model = Model(n_input_features=6)
optimizer = torch.optim.SGD(model.parameters(),lr=0)

In [16]:
model.load_state_dict(checkpoint["model_state"])
optimizer.load_state_dict(checkpoint["optim_state"])

optimizer.state_dict()

{'state': {},
 'param_groups': [{'lr': 0.01,
   'momentum': 0,
   'dampening': 0,
   'weight_decay': 0,
   'nesterov': False,
   'maximize': False,
   'params': [0, 1]}]}

# How to do it on GPU

In [17]:
# 1) Save on GPU, Load on CPU

"""
device = torch.device("cuda")
model.to(device)
torch.save(model.state_dict(), PATH)
"""

'\ndevice = torch.device("cuda")\nmodel.to(device)\ntorch.save(model.state_dict(), PATH)\n'

In [18]:

"""
device = torch.device('cpu')
model = Model(*args, **kwargs)
model.load_state_dict(torch.load(PATH, map_location=device))
"""

"\ndevice = torch.device('cpu')\nmodel = Model(*args, **kwargs)\nmodel.load_state_dict(torch.load(PATH, map_location=device))\n"

In [19]:
# 2) Save on GPU, Load on GPU

"""
device = torch.device("cuda")
model.to(device)
torch.save(model.state_dict(), PATH)
"""

'\ndevice = torch.device("cuda")\nmodel.to(device)\ntorch.save(model.state_dict(), PATH)\n'

In [20]:
"""
model = Model(*args, **kwargs)
model.load_state_dict(torch.load(PATH))
model.to(device)

# Note: Be sure to use the .to(torch.device('cuda')) function 
# on all model inputs, too!
"""

"\nmodel = Model(*args, **kwargs)\nmodel.load_state_dict(torch.load(PATH))\nmodel.to(device)\n\n# Note: Be sure to use the .to(torch.device('cuda')) function \n# on all model inputs, too!\n"

In [21]:
# 3) Save on CPU, Load on GPU

"""
torch.save(model.state_dict(), PATH)
"""

'\ntorch.save(model.state_dict(), PATH)\n'

In [22]:
"""

device = torch.device("cuda")
model = Model(*args, **kwargs)
model.load_state_dict(torch.load(PATH, map_location="cuda:0"))  # Choose whatever GPU device number you want
model.to(device)
# This loads the model to a given GPU device. 
# Next, be sure to call model.to(torch.device('cuda')) to convert the model’s parameter tensors to CUDA tensors
"""

'\n\ndevice = torch.device("cuda")\nmodel = Model(*args, **kwargs)\nmodel.load_state_dict(torch.load(PATH, map_location="cuda:0"))  # Choose whatever GPU device number you want\nmodel.to(device)\n# This loads the model to a given GPU device. \n# Next, be sure to call model.to(torch.device(\'cuda\')) to convert the model’s parameter tensors to CUDA tensors\n'